In [0]:
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "AKIA5**********")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "Uosypr/D****************WMiikEXO2Ak")
raw_data = spark.read.json("s3://my-bucket-om/capstone/claims.json")
#raw_data.write.format("delta").mode("overwrite").save("/mnt/delta/raw_data111")
#dbutils.notebook.exit("/mnt/delta/raw_data111")
raw_data.show()
raw_data.printSchema()

raw_data.createOrReplaceTempView("raw_data")

+-----------------+----------+------------+----------+--------+----------------+----------------+----------+
|Claim_Or_Rejected|    SUB_ID|claim_amount|claim_date|claim_id|      claim_type|    disease_name|patient_id|
+-----------------+----------+------------+----------+--------+----------------+----------------+----------+
|                N| SUBID1000|       79874|1949-03-14|       0| claims of value|    Galactosemia|    187158|
|              NaN|SUBID10001|      151142|1970-03-16|       1|claims of policy|  Bladder cancer|    112766|
|              NaN|SUBID10002|       59924|2008-02-03|       2| claims of value|   Kidney cancer|    199252|
|              NaN|SUBID10003|      143120|1995-02-08|       3|  claims of fact|         Suicide|    133424|
|                Y|SUBID10004|      168634|1967-05-23|       4| claims of value|    Food allergy|    172579|
|              NaN|SUBID10005|       64840|1991-10-04|       5|claims of policy|        Whiplash|    171320|
|                N|

In [0]:
raw_data.write.format("redshift").option("url", "jdbc:redshift://default-workgroup.905418268138.us-east-2.redshift-serverless.amazonaws.com:5439/dev").\
   option("dbtable", "test.rawdataTest").\
   option("aws_iam_role", "arn:aws:iam::**********:role/redshift_admin").\
   option("tempdir", "s3a://glue-bucket-om999/temp2/").\
   option("user", "admin").\
   option("password", "***********").save()

In [0]:
%sql
WITH DiseaseCounts AS (
    SELECT 
        disease_name, 
        COUNT(disease_name) AS totalCount 
    FROM 
        raw_data 
    GROUP BY 
        disease_name
),
MaxCount AS (
    SELECT 
        MAX(totalCount) AS maxCount 
    FROM 
        DiseaseCounts
)
SELECT 
    A.disease_name 
FROM 
    DiseaseCounts A
JOIN 
    MaxCount B 
ON 
    A.totalCount = B.maxCount;


disease_name
Pet allergy
Anthrax
Galactosemia
Glaucoma
Phenylketonuria
Head banging


In [0]:
output = spark.sql("WITH DiseaseCounts AS (SELECT disease_name, COUNT(disease_name) AS totalCount FROM raw_data GROUP BY disease_name),\
MaxCount AS (SELECT MAX(totalCount) AS maxCount FROM DiseaseCounts)\
SELECT A.disease_name FROM DiseaseCounts A JOIN MaxCount B ON A.totalCount = B.maxCount;")

output.show()

+---------------+
|   disease_name|
+---------------+
|    Pet allergy|
|        Anthrax|
|   Galactosemia|
|       Glaucoma|
|Phenylketonuria|
|   Head banging|
+---------------+



In [0]:
output.write.format("redshift").option("url", "jdbc:redshift://default-workgroup.905418268138.us-east-2.redshift-serverless.amazonaws.com:5439/dev").\
   option("dbtable", "capstone.maxClaimsforDisease").\
   option("aws_iam_role", "arn:aws:iam::**********:role/redshift_admin").\
   option("tempdir", "s3a://glue-bucket-om999/temp2/").\
   option("user", "admin").\
   option("password", "***********").save()